In [1]:
!pipenv install langchain_community

Loading .env environment variables...
Installing langchain_community...
✔ Installation Succeeded
Installing dependencies from Pipfile.lock (92ea9b)...
All dependencies are now up-to-date!
Upgrading langchain_community in  dependencies.
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠹ Locking packages...
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠸ Locking packages...
Installing dependencies from Pipfile.lock (72987a)...
All dependencies are now up-to-date!
Installing dependencies from Pipfile.lock (72987a)...


In [23]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('GROQ_API_KEY')
# print(api_key)

In [48]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model = "llama-3.1-8b-instant",
    temperature = 0,
    max_tokens = None,
    timeout = None,
    max_retries = 2,
    # otherparameters
)

In [49]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader('https://meinestadt.jobs.personio.de/job/2373058?language=en&display=en')
page_data = loader.load().pop().page_content

In [50]:
print(type(page_data))

<class 'str'>


In [51]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """
    I will give you scraped text from the job posting. 
    Your job is to extract the job details & requirements in a JSON format containing the following keys: 'role', 'experience', 'skills', and 'description'. 
    Only return valid JSON. No preamble, please.
    Here is the scraped text: {page_data}
    """
)
chain = prompt | llm
portfolio_content = chain.invoke(input={"page_data":page_data})
print(type(portfolio_content.content))
print(portfolio_content.content)

<class 'str'>
{
  "role": "Senior Backend Engineer (PHP)",
  "experience": "Proven experience as a backend engineer, with deep expertise in Laravel. Strong track record in building scalable web applications and APIs.",
  "skills": "Laravel, problem-solving, collaboration, mentoring, technical excellence, business-oriented solutions",
  "description": "Design, build, and maintain reliable, scalable backend systems with Laravel. Collaborate closely with product managers, frontend engineers, and other backend developers to deliver impactful features. Mentor and support junior and mid-level engineers, fostering a culture of growth and collaboration."
}


In [59]:
from langchain_core.output_parsers import JsonOutputParser
json_output = JsonOutputParser().parse(portfolio_content.content)
print(type(json_output))

<class 'dict'>


In [72]:
import csv
data = []
with open('senior_backend_engineer_php.csv', 'r') as file:
    csv_f = csv.reader(file)
    # print(csv_f)
    next(csv_f)
    
    for row in csv_f:
        skills = tuple(row[:-1])
        project_links = row[-1]

        data.append((skills,project_links))

for skills, project_links in data:
    print(skills, project_links)

('Laravel',) https://github.com/user/laravel-api
('Problem-solving',) https://github.com/user/php-web-app
('Collaboration',) https://github.com/user/scalable-api-laravel
('Mentoring',) https://github.com/user/full-stack-laravel
('Technical excellence',) https://github.com/user/collaborative-php-project
('Business-oriented solutions',) https://github.com/user/business-oriented-laravel


In [82]:
import chromadb
import uuid
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection("portfolio_collection")

if not collection.count():
    for skills, links in data:
        collection.add(
            documents = str(skills),
            metadatas = {'portfolio_url':links},
            ids = [str(uuid.uuid4())]
        )



In [90]:
portfolio_urls = collection.query(query_texts=json_output['skills'], n_results=2)
portfolio_urls

{'ids': [['d59efe88-29fe-43b0-b240-c1b1b7265298',
   'a350953a-358e-473b-ad08-a6ec2380df14']],
 'embeddings': None,
 'documents': [["('Laravel',)", "('Business-oriented solutions',)"]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'portfolio_url': 'https://github.com/user/laravel-api'},
   {'portfolio_url': 'https://github.com/user/business-oriented-laravel'}]],
 'distances': [[0.9111949801445007, 0.9231025576591492]]}

In [95]:
email_prompt = PromptTemplate.from_template(
    """
    I will give you a role and a task that you have to perform in that specific role.
    Your Role: Your name is Hassan, You are an incredible business development officer who knows how to get clients. You work for X Consulting firm, your firm works with all sorts of IT clients and provide solutions in the domain of Data Science and AI. 
    X AI focuses on efficient tailored solutions for all clients keeping costs down. 
    Your Job: Your Job is to write cold emails to clients regarding the Job openings that they have advertised. Try to pitch your clients with an email hook that opens a conversation about a possibility of working with them. Add the most relevant portfolio URLs from
    the following (shared below) to showcase that we have the right expertise to get the job done. 
    I will now provide you with the Job description and the portfolio URLs:
    JOB DESCRIPTION: {job_description}
    ------
    PORTFOLIO URLS: {portfolio_urls}
    """
)
chain_email = email_prompt | llm
response = chain_email.invoke({'job_description':json_output['description'], 'portfolio_urls':portfolio_urls})
print(response.content)

Here's a cold email that I, Hassan from X Consulting firm, would send to a potential client:

Subject: Expert Backend Development Solutions for Your Business

Dear [Client's Name],

I came across your job posting for a Backend Developer to design, build, and maintain reliable, scalable backend systems with Laravel. At X Consulting firm, we specialize in providing tailored solutions for businesses like yours, and I believe our expertise could be a great fit for your needs.

Our team has extensive experience in developing business-oriented solutions using Laravel, as showcased in our portfolio: https://github.com/user/laravel-api and https://github.com/user/business-oriented-laravel. These projects demonstrate our ability to deliver scalable, efficient, and reliable backend systems that meet the demands of modern businesses.

What sets us apart is our focus on collaboration and growth. We believe in mentoring and supporting junior and mid-level engineers to foster a culture of growth and